# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: Charon, Tang Wanzhen

Date: 2025/8/21

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [12]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('Alpha_API')))

ALPHAVANTAGE_API_KEY loaded? True


## Helpers (use or modify)

In [16]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [17]:
SYMBOL = 'AAPL'
USE_ALPHA = bool(os.getenv('Alpha_API'))
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_DAILY','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('Alpha_API')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    #print(js)
    key = [k for k in js if 'Time Series' in k][0]
    df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','4. close':'close'})[['date','close']]
    df_api['date'] = pd.to_datetime(df_api['date']); df_api['close'] = pd.to_numeric(df_api['close'])
else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Adj Close']]
    df_api.columns = ['date','adj_close']

v_api = validate(df_api, ['date','close']); v_api

{'missing': [], 'shape': (100, 2), 'na_total': 0}

In [18]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

Saved data/raw/api_alpha_AAPL_20250822-1835.csv


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [25]:
SCRAPE_URL = 'https://www.investing.com/rates-bonds/u.s.-1-year-bond-yield-historical-data'  # TODO: replace with permitted page
headers = {'User-Agent':'AFE-Homework/1.0'}
try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)
except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape

{'missing': [], 'shape': (48, 6), 'na_total': 51}

In [26]:
_ = save_csv(df_scrape, prefix='scrape', site='investing', table='BondYield')

Saved data/raw/scrape_investing_BondYield_20250822-1907.csv


## Documentation
- API Source: (URL/endpoint/params)
    * api - alpha Vantage
  
- Scrape Source: (URL/table description)
    * URL - 'https://www.investing.com/rates-bonds/u.s.-1-year-bond-yield-historical-data'
  
- Assumptions & risks: (rate limits, selector fragility, schema changes)
    * assuming that the columns in API stay the same in the future
    * risk includes the denial of accessing from url
  
- Confirm `.env` is not committed.